In [2]:
from Includes import *
import matplotlib
import pandas
import sqlite3


In [ ]:
conn = sqlite3.connect("databases/indices.db")
cur =  conn.cursor()

# df = pandas.read_sql("SELECT name FROM sqlite_master WHERE type = 'table'", conn)
# print(df.head(50))

def add_sma(df, window):
    df[f'sma_{window}'] = df['close'].rolling(window).mean()
    return df

def add_ema(df, window):
    df[f'ema_{window}'] = df['close'].ewm(span=window).mean()
    return df

def add_volatility(df, window):
    df[f'vol_{window}'] = df['close'].pct_change().rolling(window).std()
    return df

def add_relative_strength(df, benchmark_df, base='nifty50', window=20):
    merged = df[['date', 'close']].merge(
        benchmark_df[['date', 'close']],
        on='date',
        suffixes=('', '_bench')
    )
    merged['norm'] = merged['close'] / merged['close'].iloc[0] * 100
    merged['norm_bench'] = merged['close_bench'] / merged['close_bench'].iloc[0] * 100
    merged['RS'] = merged['norm'] / merged['norm_bench']
    merged['RS_mom'] = merged['RS'] / merged['RS'].shift(window) - 1
    return merged

sector = pandas.read_sql("SELECT * FROM banknifty", conn)
bench  = pandas.read_sql("SELECT * FROM nifty50", conn)

df = add_relative_strength(sector, bench, window=20)
df = add_sma(df, 20)
df = add_sma(df, 50)
df = add_volatility(df, 20)


           date   close_asset   close_index  asset_ret  index_ret  \
4186 2025-11-27  59737.300781  26215.550781   0.023157   0.006204   
4187 2025-11-28  59752.699219  26202.949219   0.029667   0.012563   
4188 2025-12-01  59681.351562  26175.750000   0.032972   0.017637   
4189 2025-12-02  59273.800781  26032.199219   0.020178   0.010435   
4190 2025-12-03  59348.250000  25986.000000   0.026306   0.015171   

      rs_vs_nifty50  rs_ratio_vs_nifty50  
4186       0.016953             1.016848  
4187       0.017104             1.016892  
4188       0.015335             1.015070  
4189       0.009742             1.009642  
4190       0.011135             1.010968  


In [ ]:
conn.close()